# Import

In [1]:
import numpy as np
import random as rd

# Define

In [2]:
class VariableMatrix():
    def __init__(self, size, work_components = 1):
        self.values = np.zeros(size)
        self.diff = np.zeros(size)
        if work_components == 1:
            self.work = np.ones(size, dtype = bool)
        else:
            self.work = np.zeros(size, dtype = bool)
            for i in range(size[0]):
                for j in range(size[1]):
                    self.work[i][j] = rd.random() < work_components
    
    def random_value(self, interval = [-1, 1]):
        for i in range(self.values.shape[0]):
            for j in range(self.values.shape[1]):
                self.values[i][j] = (interval[1] - interval[0]) * rd.random() + interval[0] 
                    
class Sigmoid():
    def trans(self, x):
        out_put = np.zeros([len(x)])
        for i in range(len(x)):
            out_put[i] = 1 / (1 + np.exp(-x[i]))
        
        return out_put
    
    def diff(self, x):
        out_put = np.zeros([len(x), len(x)])
        for i in range(len(x)):
            out_put[i][i] = 1 / (1 + np.exp(x[i])*(1 + np.exp(-x[i])))
            
        return out_put
    
class Identity():
    def trans(self, x):
        return x
    
    def diff(self, x):
        return np.identity(len(x))

class Layer():
    def __init__(self, layers_from, nodes_n, active_function):
        self.lf = layers_from # list :layers connected before
        self.lt = [] # layers connected after
        self.nodes_n = nodes_n
        self.weights = {}
        self.w_diff = {}
        self.bias = np.zeros([nodes_n])
        self.b_diff = np.zeros([nodes_n])
        self.nodes_before = np.zeros([nodes_n])
        self.nodes_after = np.zeros([nodes_n])
        self.nodes_diff = np.zeros([nodes_n])
        self.af = active_function
    
    def forward(self):
        self.nodes_after = self.af.trans(self.nodes_before)
    
    def jacobi(self):
        self.nodes_diff = self.af.diff(self.nodes_before)
    
class DogikoNN():
    def __init__(self):
        self.training_data = None
        self.pp_linear = {
            "multi" : None,
            "trans" : None
        } # pre-processing linear translation
        self.flow = []
        self.layers = {}
    
    def set_data(self, input_data, input_answer, input_weights = None):
        # type input_data : numpy array
        # type input_answer : numpy array
        self.t_datas = input_data
        self.t_answers = input_answer
        self.t_weights = input_weights
        
    def define_normalized(normalized_algorithm):
        self.normal_alg = normalized_algorithm
    
    def layers_clear():
        del self.layers
        self.layers = {}
        
    def insert_layer(name, layer, is_flow_add = True):
        self.layers[name] = layer
        if is_flow_add:
            self.flow.append(name)
        
    def build():
        if self.normal_alg == "normal":
            temp_trans_datas = self.datas.T
            temp_sqre_mean_array = np.zeros([len(temp_trans_datas)])
            self.pp_linear["trans"] = np.zeros([len(temp_trans_datas)])
            for j in range(len(temp_trans_datas)):
                feature = temp_trans_datas[j]
                self.pp_linear["trans"][j] = feature.mean
                temp_square_mean_array[j] = (feature**2).mean
            
            self.pp_linear["multi"] = (temp_square_mean_array - self.pp_linear["trans"]**2)**0.5
        
        else:
            print ("unacceptable normalized_algorithm")
            return ("unacceptable normalized_algorithm")
        
        self.n_t_datas = np.zeros()
        for i in range(len(self.t_datas)):
            self.n_t_datas[i] = self.pp_linear["multi"] * (self.t_datas[i] - self.pp_linear["trans"])
        
        if len(self.flow) != len(set(self.flow)):
            print ("each element in self.flow should be unique (layer name)")
            return ("each element in self.flow should be unique (layer name)")
        
        for l in range(len(self.flow)):
            ln = self.flow[l]
            for ln2 in self.layers[ln].lf:
                if self.flow.index(ln2) > l:
                    print ("layer " + ln2 + " shoud work before layer " + ln)
                    return ("layer " + ln2 + " shoud work before layer " + ln)
                    
                self.layers[ln2].lt.append(ln)
                self.layers[ln].weights[ln2] = np.zeros([self.layers[ln].nodes_n, self.layers[ln2].nodes_n])
                self.layers[ln].w_diff[ln2] = np.zeros([self.layers[ln].nodes_n, self.layers[ln2].nodes_n])